# Baseline - [Data Train] Competição ML #1 - Titanic

### Para o notebook completo seguindo o método CRISP-DM acesse: https://github.com/wandersondsm/Competicao-ML-1---Data-Train

### Espero que todos se divirtam e assim possamos aprender juntos!
### Para troca de ideias e discursões sobre a competição, entre na comunidade no telegram: https://t.me/thedatatrain


#### Autor: Wanderson Marques - wdsmarques@gmail.com

### Listagem os arquivos armazenados na sessão do kaggle

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Todos os imports necessários para o projeto

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc



### Configuração de parâmetros

In [ ]:
# Exibir gráficos dentro do Jupyter Notebook
%matplotlib inline

# Definir tamanho padrão para os gráficos
rcParams['figure.figsize'] = 17, 4

### Carregando o dataset de treino

In [ ]:
dataset_original = pd.read_csv('../input/data-train-competicao-ml-1-titanic/train.csv')

# Eliminar o identificador dos passageiros
dataset = dataset_original.drop(['PassengerId'], axis=1)

### Análise exploratória para conhecer o dataset

In [ ]:
dataset.head()

In [ ]:
dataset.dtypes

In [ ]:
# [0] = Quantidade de instâncias
# [1] = Quantidade de atributos
print("O dataset contém ", dataset.shape[0], "instâncias e ", dataset.shape[1], " atributos.")

In [ ]:
# .sample() Mostra uma amostra aleatória
# .head() Mostra as primeiras instâncias
# .tail() Mostra as últimas instâncias
dataset.sample(5)

In [ ]:
# Somente atributos numéricos são considerados
dataset.describe()

In [ ]:
# Quantidade absoluta
totalNulos = (dataset.isnull()).sum()
totalNulos

In [ ]:
# Percentual
percentualNulos = (totalNulos / len(dataset)) * 100
percentualNulos

In [ ]:
# Apenas atributos int e float
numericos = (dataset.select_dtypes(include=['int64', 'float64'])).columns

In [ ]:
# Apenas atributos object (string)
categoricos = (dataset.select_dtypes(include=['object'])).columns

# Não considerar os atributos textuais Name, Ticket e Cabin entre os atributos categóricos 
categoricos = categoricos.drop(['Name', 'Ticket', 'Cabin'])

In [ ]:
fig, ax = plt.subplots(ncols=len(numericos), nrows=1)
plt.suptitle("Boxplots dos Atributos Numéricos")

# Gráfico para cada atributo numérico
for i in range(0, len(numericos)):
    feature = numericos[i]
    sns.boxplot(dataset[feature], ax=ax[i], orient='vertical')

Os boxplots acima mostra ainda mais claramente a existência de outliers para Age, SibSp, Parch e Fare, principalmente para as 3 últimas. Em etapas posteriores, pode ser que seja necessário realizar algum tratamento nas instâncias que apresentam esse tipo de comportamento.

In [ ]:
fig, ax = plt.subplots(ncols=len(numericos), nrows=1)
plt.suptitle("Histogramas dos Atributos Numéricos")

# Histograma para cada atributo numérico
for i in range(0, len(numericos)):
    feature = numericos[i]
    ax[i].set_title(feature)
    dataset[feature].plot(kind='hist', ax=ax[i])

Analisando as distribuições, temos que:

    Existiam mais passageiros na terceira classe do que nas outras 2
    A variável Age é a que mais se aproxima de uma Distribuição Normal
    SibSp, Parch e Fare possuem, em sua maioria, valores pequenos

In [ ]:
fig, ax = plt.subplots(ncols=len(categoricos), nrows=1)
plt.suptitle("Gráficos de Barra dos Atributos Categóricos")

# Gráfico para cada atributo categórico
for i in range(0, len(categoricos)):
    feature = categoricos[i]
    ax[i].set_title(feature)
    dataset[feature].value_counts().plot(kind='bar', ax=ax[i])

Existiam mais homens do que mulheres a bordo e a maioria dos passageiros embarcaram em "S" (Southampton)


In [ ]:
# Somente atributos numéricos são considerados
plt.suptitle("Gráfico de Calor das Correlações entre os Atributos Numéricos")
sns.heatmap(dataset.corr(), annot=True, cmap='Blues')

O gráfico de calor mostra que as variáveis mais correlacionadas são Pclass e Fare (negativamente) e SibSp e Parch (positivamente).

## Pré-Processamento

In [ ]:
dataset_original.shape

In [ ]:
# removendo features que não serão utilizadas no treinamento
dataset = dataset_original.drop(['Name', 'Ticket', 'Cabin', 'PassengerId', 'Embarked', 'Parch', 'Fare'], axis=1)
dataset.head()

In [ ]:
# divisão do dataset em treino e teste
train, test_split = train_test_split(dataset.copy(), test_size=0.3)

In [ ]:
train.isnull().sum()

In [ ]:
# Para as instâncias onde Age é nulo, imputar a média (29.97)
train.loc[train['Age'].isnull(), 'Age'] = train.mean()['Age']




In [ ]:
#Codificando a variável sexo
train['Sex'] = train['Sex'].replace(['male'], '1')
train['Sex'] = train['Sex'].replace(['female'], '0')

#transformando a variável Classe em categórica
train['Pclass'] = train['Pclass'].astype('category')



In [ ]:
train.isnull().sum()

In [ ]:
# separando o target do treinamento
X = train.drop(['Survived'], axis=1)
y = train['Survived']

In [ ]:
#treinamento do modelo com Rede Neural
model = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(4, 2), random_state=1)

model.fit(X, y)
MLPClassifier(alpha=1e-05, hidden_layer_sizes=(4, 2), random_state=1,solver='lbfgs')



### Avaliação do modelo com o dataframe de testes

In [ ]:
y_pred = model.predict(X)

In [ ]:
# acurácia
accuracy_score(y, y_pred)

In [ ]:
print(classification_report(y, y_pred))

In [ ]:
# Imputar valores nulos usando padrões do conjunto de treino
test_split.loc[dataset['Age'].isnull(), 'Age'] = 29.97
#codificando a variavel sexo
test_split['Sex'] = test_split['Sex'].replace(['male'], '1')
test_split['Sex'] = test_split['Sex'].replace(['female'], '0')
#transformando a variável Classe em categórica
test_split['Pclass'] = test_split['Pclass'].astype('category')

In [ ]:
test_split.head()

In [ ]:
X_test = test_split.drop(['Survived'], axis=1)
y_test = test_split['Survived']


In [ ]:
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)

In [ ]:
# gerando a matriz confusão
cm = confusion_matrix(y_test, y_pred)

sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
plt.title('Matriz de Confusão')
plt.ylabel('True label')
plt.xlabel('Predicted label')

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
fp, tp, thresholds = roc_curve(y_test, y_proba[:, 1])

In [ ]:
# Gerando a Curva ROC
plt.plot(fp, tp)

plt.plot([0, 1], [0, 1], '--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])

plt.xlabel('Falso Positivo')
plt.ylabel('Verdadeiro Positivo')
plt.title('Curva ROC')

In [ ]:
# Área sob a curva ROC
auc(fp, tp)

### Predição com dataset de teste para submissão na competição

In [ ]:
validation = pd.read_csv('../input/data-train-competicao-ml-1-titanic/test.csv')
validation.head()

In [ ]:
# Descartar colunas textuais e identificador
identificador = validation[['PassengerId']]
validation.drop(['Name', 'Ticket', 'Cabin', 'PassengerId', 'Embarked', 'Parch', 'Fare'], axis=1, inplace=True)

# Imputar valores nulos usando padrões do conjunto de treino
validation.loc[validation['Age'].isnull(), 'Age'] = 29.97
validation['Sex'] = validation['Sex'].replace(['male'], '1')
validation['Sex'] = validation['Sex'].replace(['female'], '0')
validation['Pclass'] = validation['Pclass'].astype('category')


In [ ]:
y_pred = model.predict_proba(validation)
y_pred = y_pred[:, 1]

In [ ]:
resultado = pd.concat([identificador, pd.DataFrame(y_pred, columns=['Survived'])], axis=1)
resultado.head()

In [ ]:
# gerando arquivos para submissão na competição
resultado.to_csv('submission.csv', index=False)